In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os
from xgboost import XGBClassifier
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import csv 
pd.options.mode.chained_assignment = None

np.random.seed(31415)
test_file = pd.read_csv("/kaggle/input/titanic/test.csv")
#print(test_file.shape) #(418,11)

train_file = pd.read_csv("/kaggle/input/titanic/train.csv")
#print(train_file.shape) #(891,12)
print(train_file.describe())
#print(train_file.columns)
samples = train_file[['Pclass', 'Name','Sex', 'Age', 'SibSp',
       'Parch', 'Ticket','Fare']] #excluding embarked and cabin, many nans and passID

#Preparing data
#print(samples['Age'].value_counts(dropna=False)) 
samples["Age"].fillna(
    samples.groupby(["Sex"])["Age"].transform(
        lambda x: x.mean()
    ), inplace=True)
#print(samples['Age'])

samples.loc[samples['Name'].notnull(), 'NameLen'] = samples['Name'].str.len()

count = 0
for idx in samples['Name']: #turn name into title, more meaningful
    idx = idx.split(",")[1].split(".")[0]
    #print(idx)
    samples.iloc[count,1] = idx
    #print(samples.iloc[count,1])
    count=count+1
                  
#samples.loc[samples['Cabin'].notnull(), 'Cabin'] = samples['Cabin'].str[0] #replace cabin with its initial letter
#cond = (samples['Sex'] == female)
#result_female_ages = samples[cond].column_3.values[0]

#samples.loc[samples['Embarked'].isna(), 'Embarked'] = "S"
#print(samples.iloc[:,:].head(10))

samples = pd.get_dummies(samples, prefix=['Sex', 'Ticket','Name'],
                         columns=['Sex','Ticket','Name']) #one-hot encoding
samples = np.array(samples)

labels = train_file[ 'Survived']
#print(samples.shape, labels.shape)
labels = np.array(labels)

#train_features, test_features, train_labels, test_labels = train_test_split(samples, labels, test_size = 0.25, random_state = 42)

#random forest classifier model
clf = RandomForestClassifier()
#xgb = XGBClassifier()

parameters = {
    "n_estimators":[5,10,50,100,250],
    "max_depth":[2,4,8,16,32,None]
}

#grid search of best parameters and kfold cross validation
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(clf,parameters,cv=10)
cv.fit(samples,labels)


def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

best_RF_model = RandomForestClassifier(n_estimators = 250)

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max   

/tmp/ipykernel_21/2863829748.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  samples.loc[samples['Name'].notnull(), 'NameLen'] = samples['Name'].str.len()


Best parameters are: {'max_depth': None, 'n_estimators': 250}


0.703 + or -0.069 for the {'max_depth': 2, 'n_estimators': 5}
0.691 + or -0.066 for the {'max_depth': 2, 'n_estimators': 10}
0.691 + or -0.04 for the {'max_depth': 2, 'n_estimators': 50}
0.675 + or -0.044 for the {'max_depth': 2, 'n_estimators': 100}
0.662 + or -0.036 for the {'max_depth': 2, 'n_estimators': 250}
0.73 + or -0.07 for the {'max_depth': 4, 'n_estimators': 5}
0.791 + or -0.049 for the {'max_depth': 4, 'n_estimators': 10}
0.778 + or -0.038 for the {'max_depth': 4, 'n_estimators': 50}
0.787 + or -0.038 for the {'max_depth': 4, 'n_estimators': 100}
0.787 + or -0.03 for the {'max_depth': 4, 'n_estimators': 250}
0.79 + or -0.037 for the {'max_depth': 8, 'n_estimators': 5}
0.799 + or -0.042 for the {'max_depth': 8, 'n_estimators': 10}
0.811 + or -0.037 for the {'max_depth': 8, 'n_estimators': 50}
0.811 + or -0.028 for the {'max_depth': 8, 'n_estimators': 100}
0.811 + or -0.033 for the {'max_depth': 8, 'n_estimators'